# Computational Systems Biology
# HW2

In this homework, you study different kinds of reaction couplings in a metabolic network. Regardless of the biological intuition behind the couplings, these relations help us speed up exponential-time MILP problems in metabolic networks, including the most important problems in this field. After that, you will use QFCA to find a quantitative equation for a flux according to a proper set of irreversible fluxes.

In [ ]:
import cobra

In this HW, you are allowed to use built-in cobra functions for finding blocked and essential reactions, which are `cobra.flux_analysis.find_blocked_reactions(model)` and `cobra.flux_analysis.find_essential_reactions(model)`, respectively. The first one returns a list of ids, but the second one returns a list of Reaction objects!

To define new variables to the model other then reactions, you can use `new_var = model.problem.Variable(name, lb=None, ub=None, type='continuous')`. The variable type can be `'continuous'`, `'integer'`, or `'binary'`. To add your variable to the model, use `model.add_cons_vars([new_var])`. Note that to use these variables in taylored constraints, you should use them without any `.flux_expression`. In fact, for a `cobra.Reaction` object, `reaction.flux_expression` returns the Variable related to that object, but here we are using a `Variable` object directly.

# Task 1: Coupling

For each pairs of reactions, $j_1$ and $j_2$, five possible coupling cases is imaginable:
1. Fully Coupled ($v_1 \Leftrightarrow v_2$): $R_{min} = R_{max} = c > 0$
2. Directionally Coupled ($v_1 \rightarrow v_2$): $R_{min} = 0, R_{max} = c > 0$
3. Directionally Coupled ($v_1 \leftarrow v_2$): $R_{min} = c > 0, R_{max} = \infty$
4. Partially Coupled ($v_1 \leftrightarrow v_2$): $R_{min} = c_1 > 0, R_{max} = c_2 > 0$
5. Uncoupled: $R_{min} = 0, R_{max} = \infty$

for $R = \frac{v_{j_1}}{v_{j_2}}$ and $J$ is the set of reactions. Finding $R_{min}$ and $R_{max}$ is based on solving the following optimization problem (a.k.a. FCF):

$$
minimize/maximize  \quad R = \frac{v_{j_1}}{v_{j_2}} \\
s.t.: \qquad \quad Sv = 0 \\
\qquad \qquad \qquad \qquad l_j \leq v_j \leq u_j \qquad j \in J \\
\qquad \qquad \qquad \qquad v_j \geq 0 \qquad j \in J.
$$ 

Note that in this formulation, all reversible reactions ($A \leftrightarrow B$) are replaced by two irreversible reactions 
($A \rightarrow B$ and $A \leftarrow B$) to prevent analytical problems for the fraction $\frac{v_{j_1}}{v_{j_2}}$. It would be unnecessary if you handle this concern differently.

## Part 1:

By transforming this problem to an LP, find all the irreversible reactions directionally coupled to biomass reaction (i.e., biomass $\rightarrow v_j$) in our `iAF1260` E.Coli metabolic network in the aerobic medium (consider fully and partially couplings as a special case of directional coupling). Compare the result to irreversible essential reactions and save the reactions which belong to one of these sets and not the other (i.e., the essential reactions which are not directionally coupled to biomass and vice versa). Be careful to consider off_reactions in aerobic conditions like in HW1. 

**Note:** Your equivalent LP can be in a different format than an FBA problem (but it shouldn't be drastically different!). It is not compulsory to use the COBRA toolbox for solving your LP problem, but here are some hints to keep using it. You learned how to add new variables to the model (dummy reaction is also a choice). To add any constraints to your model, you can use `new_constraint = model.problem.Constraint(rxn.flux_expression, lb=min_val, ub=max_val)` (one of the `lb` or `ub` arguments could be ignored or set to `None`), then `model.add_cons_vars([new_constraint])`. Review the tutorial in HW1 for more details. If any of your reaction variables do not contain lower and upper bounds, simply set those bounds to great negative and positive values. Consider `M = 1000000` or any greater number as infinity wherever necessary.

In [ ]:
task1_model = cobra.io.read_sbml_model("iAF1260.xml")

In [ ]:
biomass_reaction = task1_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

In [ ]:
# Your Code

In [ ]:
## Save Your Results:
with open('Solutions/Task 1/task1_part1.txt', 'w') as f:
    for rxn_id in rxn_ids_list:
        f.write("%s\n" % rxn_id)

## Part 2:

Use your LP in the previous part, find all partially or fully coupled pairs in our `iAF1260` E.Coli metabolic network in the aerobic medium. You **only** need to investigate the pairs indicated in the `Desirable_Reactions.txt` file. 

**Computational Hint:** These two kinds of coupling have transitivity properties!

In [ ]:
# Your Code

Save your results in a text file with pairs like `('id_1', 'id_2')` in each line separately.

In [ ]:
## Save Your Results:
with open('Solutions/Task 1/task1_part2.txt', 'w') as f:
    for _pair in coupled_pairs_list:
        f.write("%s\n" % str(_pair))

# Task 2: QFCA


In this task, you investigate the accuracy of the DCE theorem:  

**DCE Theorem:** In a network without any irreversible blocked reaction, for an arbitrary reaction $R_j$, if $D_j$ is the non-empty set of all the irreversible reactions directionally coupled to $R_j$ (excluding itself), there exists positive coefficients $\{c_d\}_{d: R_d \in D_j}$ such that:

$$
v_j = \sum_{d: R_d \in D_j} c_d v_d
$$ 

held for all feasible flux vectors $v$.  

Considering $R_j$ to be the biomass reaction, find these coefficients for the irreversible reactions directionally coupled to the biomass reaction in the aerobic condition (don't save zero coefficients). Save your result as a dictionary with keys as reaction ids and values as coefficients.

**Hint:** Positive or negative certificates are proper for this job. Also, dual variables could help, accessible in an LP solution by `solution.shadow_prices` related to each metabolite. If necessary, you can have any desirable objective function for the FBA by using `model.objective = model.problem.Objective(expression, direction)`.

**Note:** In the basic model, the (lb, ub) for `ATPM`, `EX_glc(e)`, and `EX_o2(e)` are $(8.39, 8.39)$, $(-10, 1000)$, and $(-20, 1000)$ respectively. For other reactions, (lb, ub) are $(-1000, 1000)$ or $(0, 1000)$.

In [ ]:
task2_model = cobra.io.read_sbml_model("iAF1260.xml")

You can use this script to get a model free of blocked reactions:

In [ ]:
task2ـconsistent_model = cobra.flux_analysis.fastcc(task2_model)

In [ ]:
# Your Code

In [ ]:
## Save your result:
import json
with open('./Solutions/Task2/task2.json', 'w') as fp:
    json.dump(coefficients_dict, fp)